In [3]:
import cv2      
import os       
import sys     
import speech_recognition as sr
import moviepy.editor
import soundfile as sf
import docx


# No. of Images required
maxNumberOfImages =  1 + 10
progressBarLen = 40

videoInPath = "test1.mp4"

doc = docx.Document()


def progressBar(current, total, comment=''):
    progress = int(round(progressBarLen * current / float(total)))

    percents = round(100.0 * current / float(total), 1)
    fillerString = '=' * progress + '-' * (progressBarLen - progress)

    sys.stdout.write('[%s] %s%% %s\r' % (fillerString, percents, comment))
    sys.stdout.flush()

if len(sys.argv)==1:
    print("-s: Sourcefile, -o Output, -p max Pictures(default = 100)")
    exit(1)
else:
    lastarg = ""
    for arg in sys.argv:
        if lastarg == "-s":
            videoInPath = arg
            print(videoInPath)
        elif lastarg == "-o":
            pictureOutPath = arg
        elif lastarg == "-p":
            maxNumberOfImages = int(arg)
        
        lastarg = arg
    
if  os.path.exists(videoInPath):
    cam = cv2.VideoCapture(videoInPath)
    print("Open: " + videoInPath)
else:
    print("Video don't exists")
    exit(1)



length = int(cam.get(cv2.CAP_PROP_FRAME_COUNT))     #get amount of frames
setSkipper= int(length/maxNumberOfImages)
setSkipper+=1                                       # every setSkipper frame will be used

print(length," frames in total.")
print("Use ever " + str(setSkipper) + "frame.")

try: 
    pictureOutPath = videoInPath + "Images"
    if not os.path.exists(pictureOutPath):          #check if dir already exists
        os.makedirs(pictureOutPath) 

except OSError:                                     # if not created then raise error 
    print ('Error: Creating directory of data') 
  

currentframe = 0
skipper=0

#process Video
while(True): 
    exits,frame = cam.read()  # reading from frame 
  
    if skipper >= setSkipper:
        if exits: 
            # if video is still left continue creating images 
            name = pictureOutPath+'/frame' + str(currentframe) + '.jpg' 
            cv2.imwrite(name, frame) 
            currentframe += 1
            skipper = 0
            progressBar(currentframe,maxNumberOfImages, "create: frame"+str(currentframe-1)+".jpg")
            doc.add_picture(name, width=docx.shared.Inches(6), height=docx.shared.Inches(4))

        else: 
            break
    skipper +=1
    
progressBar(maxNumberOfImages,maxNumberOfImages,"create: frame"+str(currentframe-1)+".jpg")
print()

# Release all space and windows once done 
cam.release() 
cv2.destroyAllWindows()

print("finished")


video = moviepy.editor.VideoFileClip(videoInPath)
audio = video.audio
audio.write_audiofile(videoInPath + '.wav')

# Accureacy around 85%

r = sr.Recognizer()

har = sr.AudioFile(videoInPath + '.wav')

f = sf.SoundFile(videoInPath + '.wav')

seconds = float(format(len(f) / f.samplerate))
audio = [] # list to store chunks of audios
i = 0.00

with har as source:    
    while i <= seconds:
        # add audio chunks to list
        audio.append(r.record(source, duration = 18))
        i+=20

        
    try:
        #iterate through all elements 
        for aud in audio:
            # using google recognizer extract all text from all  chunks  "Required High internet connection"
            # printing result, you can store it in any variable and also create lyrics******
            
            vaal=r.recognize_google(aud)
            #print(vaal,end='\n')
            doc.add_paragraph(vaal)
            #ad.add_run(vaal,end='\n')

    except Exception as e:
            print(e)
           
        
        

doc.save(videoInPath + '.docx')
            



Open: test1.mp4
3560  frames in total.
Use ever 324frame.
[========================================] 100.0% create: frame9.jpg
finished
MoviePy - Writing audio in test1.mp4.wav


MoviePy - Done.
